In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
import re
import os


In [2]:
# Cargar el DataFrame
Mobilidad = pd.read_hdf('C:/Users/MARIA PAULA/Downloads/EAFIT/Proyecto de Grado/Mobilidad/Train/df_unido.h5', key='df')

In [3]:
Mobilidad.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13955525 entries, 2021-01-01 to 2022-09-30
Data columns (total 31 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Carril               category      
 1   Fecha Trafico        datetime64[ns]
 2   Hora                 int64         
 3   dia                  category      
 4   dia-num              int64         
 5   mes-num              int64         
 6   mes                  category      
 7   año                  int64         
 8   Velocidad (Km/h)     float64       
 9   Corredor             category      
 10  sentido              category      
 11  Operación            category      
 12  Intensidad           float64       
 13  Nº vehiculos long 1  float64       
 14  Nº vehiculos long 2  float64       
 15  Nº vehiculos long 3  float64       
 16  Ocupación            float64       
 17  Tipo de Subsistema   category      
 18  Longitud             float64       
 19  Latit

# Pipelines

* Leer los datos.
* Convertir columnas object a category.
* Eliminar las columnas específicas.
* Separar columnas numéricas y categóricas.
* Filtrar las comunas que estén en el dataset de calidad de aire.
* Limpiar las columnas categóricas (poner en minúsculas, eliminar símbolos y espacios).
* Eliminar filas con datos nulos en Operación.
* Codificar la columna Operación con LabelEncoder.
* Imputar datos faltantes con IterativeImputer para las columnas numéricas.
* Unir el dataset imputado con Fecha Trafico del dataset original.

In [4]:
df = Mobilidad.copy()

In [5]:
#Ordenar dataset por fechas 
df = df.sort_values(by='Fecha Trafico')

# 2. Convertir columnas tipo 'object' a 'category'
df = df.apply(lambda col: col.astype('category') if col.dtype == 'object' else col)

# 3. Eliminar columnas especificadas
columns_to_drop = [
    'Carril', 'Corredor', 'sentido', 'Tipo de Subsistema', 'Identificador (F/V)',
    'Nº vehiculos long 4', 'Intensidad (veh/h)', 'Ocupación (%)', 
    'Categoria 1 (Veh/h)', 'Categoria 2 (Veh/h)', 'Categoria 3 (Veh/h)', 
    'Categoria 4 (Veh/h)', 'nombre comuna', 'Comuna', 'dia', 'mes'
]
df = df.drop(columns=columns_to_drop)

# 4. Obtener columnas numéricas y categóricas
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['category']).columns.tolist()

# 5. Filtrar las comunas que estén en el dataset de calidad de aire [10, 16, 14, 4, 8]
calidad_aire_comunas = [10, 16, 14, 4, 8]
df = df[df['codigo comuna'].isin(calidad_aire_comunas)]

# 6. Limpiar las columnas categóricas (minúsculas, sin símbolos ni espacios)
def clean_categories(col):
    return col.str.lower().apply(lambda x: re.sub(r'[^a-z0-9]', '', str(x).strip()))

df[cat_cols] = df[cat_cols].apply(clean_categories)

# 7. Eliminar filas con datos nulos en 'Operación'
df = df.dropna(subset=['Operación'])

# 8. Separar las columnas que se usarán para imputación
imputation_cols = [col for col in df.columns if col not in ['Fecha Trafico', 'Operación']]

# 9. Crear transformaciones para las columnas
# Codificar la columna 'Operación' con OrdinalEncoder dentro del pipeline
cat_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder())  # OrdinalEncoder para las columnas categóricas
])

# 10. Crear el pipeline de imputación con IterativeImputer
num_transformer = IterativeImputer(max_iter=10, random_state=0)

# ColumnTransformer para numericas y categoricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),           # Transformaciones para columnas numéricas
        ('cat', cat_transformer, ['Operación'])       # Codificación de 'Operación'
    ])

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor)
])

# Separar la fecha para unirla más adelante
df_fecha = df[['Fecha Trafico']].copy()

# Aplicar la pipeline de preprocesamiento al DataFrame sin 'Fecha Trafico'
df_imputed = pd.DataFrame(pipeline.fit_transform(df), columns=num_cols + ['Operacion_encoded'])


# 11. Unir el dataset imputado con la columna 'Fecha Trafico'
df_final = pd.concat([df_fecha.reset_index(drop=True), df_imputed], axis=1)

# Guardar el DataFrame final en un archivo
df_final.to_hdf('data_imputed.h5', key='df', mode='w')

print("Preprocesamiento completado y guardado en 'data_imputed.h5'.")

Preprocesamiento completado y guardado en 'data_imputed.h5'.


In [6]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6160145 entries, 0 to 6160144
Data columns (total 15 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Fecha Trafico        datetime64[ns]
 1   Hora                 float64       
 2   dia-num              float64       
 3   mes-num              float64       
 4   año                  float64       
 5   Velocidad (Km/h)     float64       
 6   Intensidad           float64       
 7   Nº vehiculos long 1  float64       
 8   Nº vehiculos long 2  float64       
 9   Nº vehiculos long 3  float64       
 10  Ocupación            float64       
 11  Longitud             float64       
 12  Latitud              float64       
 13  codigo comuna        float64       
 14  Operacion_encoded    float64       
dtypes: datetime64[ns](1), float64(14)
memory usage: 705.0 MB


In [7]:
def missing_values(df):
    Column_Names = df.columns

    Total_rows = df.shape[0]

    missing_values = df.isnull().sum()

    Percent_missing = (missing_values / Total_rows)*100
    
    result = pd.DataFrame({
        'Column_Name': Column_Names,
        'Missing_Values': missing_values.values,
        'Total_Rows': Total_rows,
        'Percent_Missing': Percent_missing.values
    })

    return result

In [8]:
missing_values(df_final)

,Column_Name,Missing_Values,Total_Rows,Percent_Missing
0,Fecha Trafico,0,6160145,0.0
1,Hora,0,6160145,0.0
2,dia-num,0,6160145,0.0
3,mes-num,0,6160145,0.0
4,año,0,6160145,0.0
5,Velocidad (Km/h),0,6160145,0.0
6,Intensidad,0,6160145,0.0
7,Nº vehiculos long 1,0,6160145,0.0
8,Nº vehiculos long 2,0,6160145,0.0
9,Nº vehiculos long 3,0,6160145,0.0


In [9]:
df_final['Fecha Trafico'].unique()

<DatetimeArray>
['2021-01-01 00:00:00', '2021-01-01 01:00:00', '2021-01-01 02:00:00',
 '2021-01-01 03:00:00', '2021-01-01 04:00:00', '2021-01-01 05:00:00',
 '2021-01-01 06:00:00', '2021-01-01 07:00:00', '2021-01-01 08:00:00',
 '2021-01-01 09:00:00',
 ...
 '2023-05-31 14:00:00', '2023-05-31 15:00:00', '2023-05-31 16:00:00',
 '2023-05-31 17:00:00', '2023-05-31 18:00:00', '2023-05-31 19:00:00',
 '2023-05-31 20:00:00', '2023-05-31 21:00:00', '2023-05-31 22:00:00',
 '2023-05-31 23:00:00']
Length: 21140, dtype: datetime64[ns]